# Recipe1M + Spark

In this notebook, we will apply big data algorithms to the Recipe1M+ dataset found [here](http://pic2recipe.csail.mit.edu/)

# Downloads

In [1]:
# # Java
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [252]:
# Pyhon libary installs
# NOTE: Might need apache spark downloads, check later
# !pip install pandas

# !pip install pyspark
# !pip install spark-nlp==3.0.0

# Imports

In [253]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [254]:
import random
import json

import pandas as pd

import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vector, Vectors, VectorUDT
# from org.apache.spark.ml.linal import Vector, Vectors # Typo in John Snow Labs

from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType, DoubleType
from pyspark.ml.clustering import LDA

import pyspark.sql.functions as F
from pyspark.sql.functions import explode

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *              
from sparknlp.pretrained import PretrainedPipeline

In [255]:
# from google.colab import drive
# drive.mount('/content/drive')

In [256]:
import os
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [257]:
# file_path = '/content/drive/MyDrive/layer1.json'
file_path = 'layer1.json'

In [258]:
import sparknlp
# spark = sparknlp.start()
# for GPU training >> spark = sparknlp.start(gpu=True)
spark = sparknlp.start(gpu=True, memory="32G")

In [259]:
sparknlp.version()
spark.version

'3.1.1'

# Loading the Data

In [260]:
#Can do this with spark later maybe?

In [261]:
df1 = spark.read.json(file_path)

In [262]:
cols_to_drop1 = ("_corrupt_record", "url", "partition", "id", "instructions") #maybe drop some more later
df1 = df1.drop(*cols_to_drop1)

In [263]:
df1 = df1.dropna()

In [264]:
df1

DataFrame[ingredients: array<struct<text:string>>, title: string]

In [265]:
df1 = spark.createDataFrame(df1.head(20))

In [266]:
df1

DataFrame[ingredients: array<struct<text:string>>, title: string]

In [267]:
# with open(file_path, 'r') as f:
#     pdf = pd.read_json(f)
# pdf = pdf.head(100)
# print('LENGTH: ', len(pdf)) #1029720

In [268]:
# df = spark.createDataFrame(pdf).toDF(*pdf.columns)
# cols_to_drop = ("url", "partition", "id", "instructions") #maybe drop some more later
# df = df.drop(*cols_to_drop)
# display(df)
# df.show()

In [269]:
# # this is definitely new to me
# # why to use udf instead of map https://medium.com/@fqaiser94/udfs-vs-map-vs-custom-spark-native-functions-91ab2c154b44
# # https://stackoverflow.com/questions/29109916/updating-a-dataframe-column-in-spark

# name = 'ingredients' #taret_column name
# def parse_ingredients(ingredients):
#   # return "test"
#   return '. '.join([data['text'] for data in ingredients])

# udf = UserDefinedFunction(parse_ingredients, StringType())

# df = df.select(*[udf(column).alias(name) if column == name else column for column in df.columns])

In [270]:
# this is definitely new to me
# why to use udf instead of map https://medium.com/@fqaiser94/udfs-vs-map-vs-custom-spark-native-functions-91ab2c154b44
# https://stackoverflow.com/questions/29109916/updating-a-dataframe-column-in-spark

name = 'ingredients' #taret_column name
def parse_ingredients(ingredients):
  # return "test"
  return ', '.join([data['text'] for data in ingredients])

udf1 = UserDefinedFunction(parse_ingredients, StringType())

df1 = df1.select(*[udf1(column).alias(name) if column == name else column for column in df1.columns])

In [271]:
# name = 'instructions' #taret_column name
# def parse_instructions(instructions):
#   # return "test"
#   return ' '.join([data['text'] for data in instructions])


# udf = UserDefinedFunction(parse_instructions, StringType())

# df = df.select(*[udf(column).alias(name) if column == name else column for column in df.columns])

In [272]:
df1.show(5)

+--------------------+--------------------+
|         ingredients|               title|
+--------------------+--------------------+
|6 ounces penne, 2...|Worlds Best Mac a...|
|1 c. elbow macaro...|Dilly Macaroni Sa...|
|8 tomatoes, quart...|            Gazpacho|
|2 12 cups milk, 1...|Crunchy Onion Pot...|
|1 (3 ounce) packa...|Cool 'n Easy Crea...|
+--------------------+--------------------+
only showing top 5 rows



In [273]:
type(df1.collect()[0]['ingredients'])

str

In [274]:
documentAssembler = DocumentAssembler() \
    .setInputCol("ingredients") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")

In [275]:
# tokenizer = Tokenizer() \
#   .setInputCols("document") \
#   .setOutputCol("token")

In [276]:
# normalizer = Normalizer()\
#   .setInputCols("token") \
#   .setOutputCol("normal")\
#   .setLowercase(True)

In [277]:
# stopwords_cleaner = StopWordsCleaner()\
#     .setInputCols("normal")\
#     .setOutputCol("cleanTokens")\
#     .setCaseSensitive(False)

In [278]:
# embeddings = BertEmbeddings.pretrained("bert_base_uncased", "en") \
#       .setInputCols(["document", "cleanTokens"]) \
#       .setOutputCol("embeddings")

In [279]:
# sentence_embeddings = SentenceEmbeddings() \
#         .setInputCols(["document", "embeddings"]) \
#         .setOutputCol("sentence_embeddings") \
#         .setPoolingStrategy("AVERAGE") \
#         .setLazyAnnotator(False)

In [280]:
# Typo in the documentation: BertSentenceEmbeddings, uppercase in the E
sentence_embeddings = BertSentenceEmbeddings.pretrained() \
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings") \
    .setLazyAnnotator(False)

sent_small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]


In [281]:
# # # took time to find out I needed this finisher
embeddings_finisher = EmbeddingsFinisher() \
            .setInputCols(["sentence_embeddings"]) \
            .setOutputCols("sentence_embeddings_vectors") \
            .setOutputAsVector(True)

In [282]:
# explodeVectors = SQLTransformer()\
#     .setStatement("SELECT EXPLODE(sentence_embeddings_vectors) AS features, * FROM __THIS__")

In [283]:
# normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)

In [284]:
# vectorNormalizer = Normalizer() \
#       .setInputCol("features") \
#       .setOutputCol("normFeatures") \
#       .setP(1.0)

In [285]:
# assembler = VectorAssembler(
#     inputCols=["sentence_embeddings_vectors"],
#     outputCol="features")

In [286]:
testPipeline = Pipeline(
    stages=[
        documentAssembler, 
        sentence_embeddings,
        embeddings_finisher
 ])

In [287]:
test_model = testPipeline.fit(df1)

In [288]:
test_df = test_model.transform(df1)

In [289]:
test_df.show()

+--------------------+--------------------+--------------------+--------------------+---------------------------+
|         ingredients|               title|            document| sentence_embeddings|sentence_embeddings_vectors|
+--------------------+--------------------+--------------------+--------------------+---------------------------+
|6 ounces penne, 2...|Worlds Best Mac a...|[{document, 0, 51...|[{sentence_embedd...|       [[0.2524833381175...|
|1 c. elbow macaro...|Dilly Macaroni Sa...|[{document, 0, 24...|[{sentence_embedd...|       [[0.4110878407955...|
|8 tomatoes, quart...|            Gazpacho|[{document, 0, 30...|[{sentence_embedd...|       [[0.2620798349380...|
|2 12 cups milk, 1...|Crunchy Onion Pot...|[{document, 0, 17...|[{sentence_embedd...|       [[0.3757590055465...|
|1 (3 ounce) packa...|Cool 'n Easy Crea...|[{document, 0, 15...|[{sentence_embedd...|       [[0.4509302973747...|
|12 cup shredded c...|Easy Tropical Bee...|[{document, 0, 34...|[{sentence_embedd...|   

In [290]:
test_df.printSchema()

root
 |-- ingredients: string (nullable = true)
 |-- title: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 | 

In [292]:
# test_df.collect()[0]['sentence_embeddings_vectors']

In [293]:
type(test_df)

pyspark.sql.dataframe.DataFrame

In [294]:
# Solution to this sentence embedding to spark ml: https://github.com/JohnSnowLabs/spark-nlp/blob/b95ac300d4fe9b0c6ebdb29ef774d55e672f3067/docs/en/annotators.md#sentenceembeddings
def convertToVectorUDF(matrix):
  # return "test"
  return Vectors.dense(matrix.toArray.map(_.cast(DoubleType())))

udf_vector = UserDefinedFunction(convertToVectorUDF, DoubleType())

# df1 = df1.select(*[udf1(column).alias(name) if column == name else column for column in df1.columns])

In [295]:
list_to_vector_udf = udf(lambda l: l[0], VectorUDT())

In [296]:
df_with_vectors = test_df.select(list_to_vector_udf(test_df['sentence_embeddings_vectors']).alias("features"))

In [297]:
df_with_vectors.show()

+--------------------+
|            features|
+--------------------+
|[0.25248333811759...|
|[0.41108784079551...|
|[0.26207983493804...|
|[0.37575900554656...|
|[0.45093029737472...|
|[0.47885367274284...|
|[0.31959509849548...|
|[0.38087284564971...|
|[0.46585822105407...|
|[0.42635318636894...|
|[0.41329103708267...|
|[0.48222091794013...|
|[0.47433379292488...|
|[0.32569742202758...|
|[0.42206299304962...|
|[0.38134261965751...|
|[0.30782195925712...|
|[0.37859332561492...|
|[0.37919887900352...|
|[0.37079584598541...|
+--------------------+



In [ ]:
# this is definitely new to me
# why to use udf instead of map https://medium.com/@fqaiser94/udfs-vs-map-vs-custom-spark-native-functions-91ab2c154b44
# https://stackoverflow.com/questions/29109916/updating-a-dataframe-column-in-spark

name = 'ingredients' #taret_column name
def parse_ingredients(ingredients):
  # return "test"
  return '. '.join([data['text'] for data in ingredients])

udf1 = UserDefinedFunction(parse_ingredients, StringType())

df1 = df1.select(*[udf1(column).alias(name) if column == name else column for column in df1.columns])

In [ ]:
name = 'sentence_embeddings_vectors' #taret_column name
def list_to_vector(vector_list):
    return vector_list[0]

udf1 = UserDefinedFunction(list_to_vector, )

df1 = df1.select(*[udf1(column).alias(name) if column == name else column for column in df1.columns])

In [42]:
# test_df.select(explode("sentence_embeddings.embeddings").as("sentence_embedding"))
# .withColumn("features", convertToVectorUDF("sentence_embedding")) 
# Does not work because this is in Scala, not Python

In [119]:
# test_df.collect()[0]['bert_sentence_embeddings'][0]

In [164]:
type(test_df.collect()[0]['sentence_embeddings_vectors'][0])

pyspark.ml.linalg.DenseVector

In [95]:
name = "bert_sentence_embeddings"

a = test_df.select(*[convertToVectorUDF(column.embeddings).alias(name) if column == name else column for column in test_df.columns])

AttributeError: 'str' object has no attribute 'embeddings'

In [96]:
# b = test_df.select(explode("sentence_embeddings.embeddings").alias("sentence_embeddings")).withColumn("features", convertToVectorUDF(test_df["sentence_embeddings"]))

In [97]:
# b = test_df.withColumn("features", convertToVectorUDF(test_df["sentence_embeddings"]))k

In [99]:
c = test_df.select(list_to_vector_udf(test_df["bert_sentence_embeddings.embeddings"]).alias("features"))

In [109]:
c

DataFrame[features: vector]

In [221]:
c.printSchema()

root
 |-- features: vector (nullable = true)



In [111]:
from pyspark.ml.linalg import Vectors, SparseVector
df = spark.createDataFrame([[1, Vectors.dense([0.0, 1.0])],[2, SparseVector(2, {0: 1.0})],], ["id", "features"])

lda = LDA(k=2, seed=1, optimizer="em")

lda.setMaxIter(10)

LDA_0357877d868a

In [220]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- features: vector (nullable = true)



In [169]:
df.collect()[0]['features']

DenseVector([0.0, 1.0])

In [232]:
df.collect()[0]['features'][0]

0.0

In [223]:
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.feature import VectorAssembler

# assembler = VectorAssembler(
#     inputCols=["sentence_embeddings_vectors"],
#     outputCol="features")


# output = assembler.transform(c)

In [251]:
num_topics = 3
lda = LDA(k=num_topics, maxIter=1)
model = lda.fit(df_with_vectors)

In [226]:
num_topics = 3
lda = LDA(k=num_topics, maxIter=1)
model = lda.fit(c['features'])

TypeError: Column is not iterable

In [105]:
# .withColumn("features", convertToVectorUDF("sentence_embeddings"))

In [106]:
# a.collect()[0][5]

In [107]:
# # Now let's explode the sentence_embeddings column and have a new feature column for Spark ML
# testPipeline.select(F.explode("sentence_embeddings.embeddings").alias("sentence_embedding"))
# .withColumn("features", convertToVectorUDF("sentence_embedding"))

In [102]:
test_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|         ingredients|               title|            document|               token|          embeddings| sentence_embeddings|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|6 ounces penne. 2...|Worlds Best Mac a...|[{document, 0, 51...|[{token, 0, 0, 6,...|[{word_embeddings...|[{sentence_embedd...|[[0.0, 0.0, 0.0, ...|
|1 c. elbow macaro...|Dilly Macaroni Sa...|[{document, 0, 24...|[{token, 0, 0, 1,...|[{word_embeddings...|[{sentence_embedd...|[[0.0, 0.0, 0.0, ...|
|8 tomatoes, quart...|            Gazpacho|[{document, 0, 30...|[{token, 0, 0, 8,...|[{word_embeddings...|[{sentence_embedd...|[[0.0, 0.0, 0.0, ...|
|2 12 cups milk. 1...|Crunchy Onion Pot...|[{document, 0, 17...|[{token, 0, 0, 2,...|[{word_embeddings...|

In [103]:
test_df

DataFrame[ingredients: string, title: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, token: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, sentence_embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, features: array<array<float>>]

In [104]:
num_topics = 3
lda = LDA(k=num_topics, maxIter=1)
model = lda.fit(test_df)

IllegalArgumentException: requirement failed: Column features must be of type equal to one of the following types: [struct<type:tinyint,size:int,indices:array<int>,values:array<double>>, array<double>, array<float>] but was actually of type array<array<float>>.

In [105]:
# ---------------------------------------------------------------------------
# Py4JError                                 Traceback (most recent call last)
# <ipython-input-52-8550de25b96d> in <module>
#       1 num_topics = 3
#       2 lda = LDA(k=num_topics, maxIter=10)
# ----> 3 model = lda.fit(test_df)

# ~/miniconda3/envs/767_py3/lib/python3.9/site-packages/pyspark/ml/base.py in fit(self, dataset, params)
#     159                 return self.copy(params)._fit(dataset)
#     160             else:
# --> 161                 return self._fit(dataset)
#     162         else:
#     163             raise ValueError("Params must be either a param map or a list/tuple of param maps, "

# ~/miniconda3/envs/767_py3/lib/python3.9/site-packages/pyspark/ml/wrapper.py in _fit(self, dataset)
#     333 
#     334     def _fit(self, dataset):
# --> 335         java_model = self._fit_java(dataset)
#     336         model = self._create_model(java_model)
#     337         return self._copyValues(model)

# ~/miniconda3/envs/767_py3/lib/python3.9/site-packages/pyspark/ml/wrapper.py in _fit_java(self, dataset)
#     330         """
#     331         self._transfer_params_to_java()
# --> 332         return self._java_obj.fit(dataset._jdf)
#     333 
#     334     def _fit(self, dataset):

# ~/miniconda3/envs/767_py3/lib/python3.9/site-packages/py4j/java_gateway.py in __call__(self, *args)
#    1302 
#    1303         answer = self.gateway_client.send_command(command)
# -> 1304         return_value = get_return_value(
#    1305             answer, self.gateway_client, self.target_id, self.name)
#    1306 

# ~/miniconda3/envs/767_py3/lib/python3.9/site-packages/pyspark/sql/utils.py in deco(*a, **kw)
#     109     def deco(*a, **kw):
#     110         try:
# --> 111             return f(*a, **kw)
#     112         except py4j.protocol.Py4JJavaError as e:
#     113             converted = convert_exception(e.java_exception)

# ~/miniconda3/envs/767_py3/lib/python3.9/site-packages/py4j/protocol.py in get_return_value(answer, gateway_client, target_id, name)
#     332                     format(target_id, ".", name, value))
#     333         else:
# --> 334             raise Py4JError(
#     335                 "An error occurred while calling {0}{1}{2}".
#     336                 format(target_id, ".", name))

# Py4JError: An error occurred while calling o410.fit

In [ ]:
nlpPipeline = Pipeline(
    stages=[
        documentAssembler, 
        tokenizer,
        normalizer,
        stopwords_cleaner,
        sentence_embeddings,
        embeddings_finisher
 ])

In [ ]:
# [Row(ingredients='6 ounces penne. 2 cups Beechers Flagship Cheese Sauce (recipe follows). 1 ounce Cheddar, grated (1/4 cup). 1 ounce Gruyere cheese, grated (1/4 cup). 1/4 to 1/2 teaspoon chipotle chili powder (see Note). 1/4 cup (1/2 stick) unsalted butter. 1/3 cup all-purpose flour. 3 cups milk. 14 ounces semihard cheese (page 23), grated (about 3 1/2 cups). 2 ounces semisoft cheese (page 23), grated (1/2 cup). 1/2 teaspoon kosher salt. 1/4 to 1/2 teaspoon chipotle chili powder. 1/8 teaspoon garlic powder. (makes about 4 cups)', document=[Row(annotatorType='document', begin=0, end=514, result='6 ounces penne. 2 cups Beechers Flagship Cheese Sauce (recipe follows). 1 ounce Cheddar, grated (1/4 cup). 1 ounce Gruyere cheese, grated (1/4 cup). 1/4 to 1/2 teaspoon chipotle chili powder (see Note). 1/4 cup (1/2 stick) unsalted butter. 1/3 cup all-purpose flour. 3 cups milk. 14 ounces semihard cheese (page 23), grated (about 3 1/2 cups). 2 ounces semisoft cheese (page 23), grated (1/2 cup). 1/2 teaspoon kosher salt. 1/4 to 1/2 teaspoon chipotle chili powder. 1/8 teaspoon garlic powder. (makes about 4 cups)', metadata={'sentence': '0'}, embeddings=[])]

In [ ]:
pipline_model = nlpPipeline.fit(df)

In [ ]:
new_df = pipline_model.transform(df)

In [ ]:
new_df.show(1)

In [ ]:
# for row, elem in enumerate(new_df.limit(1).toPandas()['embeddings']):
#     print(f'[ROW BEGIN] {row}')
#     print((elem[0]))
#     print(f'[ROW END] {row}')

In [ ]:
# https://medium.com/analytics-vidhya/distributed-topic-modelling-using-spark-nlp-and-spark-mllib-lda-6db3f06a4da3
embeddings_df = new_df.select('title', 'features').limit(10)

In [ ]:
embeddings_df.show()

In [ ]:
test = embeddings_df.drop('title')

In [ ]:
num_topics = 3
lda = LDA(k=num_topics, maxIter=10)
model = lda.fit(embeddings_df)

In [ ]:
# https://github.com/JohnSnowLabs/spark-nlp/issues/2187
#  solution to mismatch of types between spark nlp and spark nlp
# https://github.com/JohnSnowLabs/spark-nlp/blob/master/src/test/scala/com/johnsnowlabs/nlp/EmbeddingsFinisherTestSpec.scala